In [4]:
print("Types and counts of stories")
df = frame.groupby(["type"]).size()
df

Types and counts of stories


type
bias            443
bs            11492
conspiracy      430
fake             19
hate            246
junksci         102
satire          146
state           121
dtype: int64

In [2]:
import numpy as np
import pandas as pd
#from pandas import DataFrame, Series
#import matplotlib.pyplot as plt

frame = pd.read_csv("fake.csv")
real_frame = pd.read_csv("all-the-news/articles1.csv")
frame.drop(['uuid', 'ord_in_thread','published','language','crawled'], axis=1, inplace=True)
real_frame.drop(['id', 'publication', 'author', 'date', 'year', 'month', 'url', 'content'], axis=1, inplace=True)
real_frame.columns = ['index', 'titles']

In [5]:
df_f = frame.fillna('No data in title field')
df_r = real_frame.fillna('No data in title field')

shape = df_f.shape
fake_category = [0] * shape[0]

shape = df_r.shape
real_category = [1] * shape[0]

df_f = df_f.filter(items=['text'])
df_r = real_frame.filter(items=['titles'])

In [6]:
#this variable is a list of titles, but some of the titles are strings and some are "nan" values from the dataframe. 
#filter out of the nan values to get the code below to work
fake_titles = df_f['text'].tolist()
real_titles = df_r['titles'].tolist()

In [7]:
import pandas as pd
import requests 
import enchant

words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words')
stoplist1 = words.text.split("\r\n")

from nltk.corpus import stopwords
stoplist2 = set(stopwords.words('english'))

stoplist1.extend(stoplist2)

fullstops = list(set(stoplist1))

def remove_stops(stoplist, wordlist):
    result = []
    for i in wordlist:
        if i not in stoplist:
                result.append(i)
    return result

def spellcheck(wordlist):
    result = []
    d = enchant.Dict("en_US")
    for i in wordlist:
        if d.check(i) or d.check(i.capitalize()):
            result.append(i)
         
    return result
def clean_text(list_of_texts):
    fully_cleaned =[]
    #normalize ocr errors
    for i in list_of_texts:
        #lowercase all
        ocr_lower = i.lower()
        #tokenize, remove punctuation and numbers, remove tabs, newlines, etc.
        ocr_cleaner = ocr_lower.replace("\n", " ").replace("\t", " ")
        ocr_tokens = ocr_cleaner.split(" ")
        
        no_numbers_or_punct = []
        for token in ocr_tokens:
            if token.isalpha():
                no_numbers_or_punct.append(token)
            else:
                
                new_token = ""
                for letter in token:
                    if letter.isalpha():
                        new_token += letter
                if new_token != "":
                    no_numbers_or_punct.append(new_token)  
        #almost_ready_before_spellcheck = remove_stops(fullstops, no_numbers_or_punct)
        #almost_ready = spellcheck(almost_ready_before_spellcheck)
        #from nltk.stem import WordNetLemmatizer
        #lemmatization
        #wordnet_lemmatizer = WordNetLemmatizer()
        #lemmas = []
        #for token in almost_ready:
        #    lemma = wordnet_lemmatizer.lemmatize(token)
        #    lemmas.append(lemma)
        
        #ready = [i for i in lemmas if len(i) > 2]
        #fully_cleaned.append(ready)
        fully_cleaned.append(no_numbers_or_punct)
    return fully_cleaned
ocr_cleaned_fake = clean_text(fake_titles)
ocr_cleaned_real = clean_text(real_titles)

In [9]:
from collections import Counter
ocr_counters_fake = [Counter(i) for i in ocr_cleaned_fake]
ocr_counters_real = [Counter(i) for i in ocr_cleaned_real]

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
v = DictVectorizer()
x_f = v.fit_transform(ocr_counters_fake)
x_r = v.fit_transform(ocr_counters_real)
y = TfidfTransformer()
z_f = y.fit_transform(x_f)
z_r = y.fit_transform(x_r)

In [10]:
scaled_vsm_f = z_f.toarray()
scaled_vsm_r = z_r.toarray()